In [21]:
# HIDE CODE
from dateutil.relativedelta import relativedelta
from joblib import dump, load

import warnings
import sys
import os
sys.path.append('/home/server/gli-data-science/')
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses

import os
import ds_db
import helper_db
from helper import transform_to_rupiah, rupiah_format

import pickle
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.1f' % x)
pd.options.mode.chained_assignment = None  # default='warn'
from IPython.display import display, HTML, display_html, IFrame
import ipywidgets as ipyw

def side_by_side_display(dfs:list, captions:list):
    output = ""
    combined = dict(zip(captions, dfs))
    styles = [
        dict(selector="caption", props=[("caption-side", "center"), ("font-size", "100%"), ("color", )])]
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline; font-size:85%' ").set_precision(2).set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0\xa0"

    display(HTML(output))

from sklearn.linear_model import LinearRegression, PoissonRegressor, Ridge, Lasso, RANSACRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor, export_text
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR

import textwrap
def split_label(list_label):
    list_label = list(list_label)
    list_label = ["<br>".join(textwrap.wrap(t, width=12)) for t in list_label ]
    return list_label

### ide kasar
> 
* daily job to loop all member alfagift 10 juta
* compute feature trx voucher used, # of trx, # of sales
* compute feature event app site open / view product / atc
* updating each feature and re predict churn feature
* lets play in month - 3 for each define trx and event feature

In [24]:
from datetime import datetime
now = datetime.now()
now_ = datetime(now.year, now.month, now.day)


q = '''

select 
    ttdr.tbtdr_created_date, 
    ttdr.tbtdr_ponta_user AS TRO_MEMBERS, 
    ttdr.tbtdr_tbto_id, 
    ttdr.tbtdr_rating AS rating
from 
    tb_transaction_delivery_rating ttdr 
where 
    ttdr.tbtdr_created_date between '2021-07-01' and '{}'
'''.format(now_)

con, cur = ds_db.connect_prd_order_4()
df_re = pd.read_sql(q, con)


con.close()



You are connected


KeyError: 'tbtdr_ponta_user'

In [26]:
df_re.head()

,tbtdr_created_date,tro_members,tbtdr_tbto_id,rating,created_date
0,2021-07-07 15:38:48.595742,9990010422084320,7113169,NaN,2021-07
1,2021-07-07 15:38:48.739876,9990013165255510,7113121,NaN,2021-07
2,2021-08-04 15:32:16.989000,9990012777522310,8415317,NaN,2021-08
3,2021-07-06 17:17:04.223000,9990013363137369,7074851,NaN,2021-07
4,2021-07-22 18:35:54.327000,9990013331138919,7740601,NaN,2021-07


In [27]:
df_re['created_date'] = df_re['tbtdr_created_date'].dt.strftime('%Y-%m')
df_re = df_re[df_re['tro_members'] != '']
df_re_g = df_re.groupby(['tro_members','created_date']).agg({'rating':'mean'})

df_re_g_u = df_re_g.unstack(level=1).fillna(0).reset_index()
df_re_g_u.columns = [' '.join(col).strip().upper() for col in df_re_g_u.columns.values]

df_re_g_u.head(20)

,TRO_MEMBERS,RATING 2021-07,RATING 2021-08,RATING 2021-09,RATING 2021-10,RATING 2021-11,RATING 2021-12
0,9990010000001200,0.0,-1.0,-1.0,-1.0,0.0,0.0
1,9990010000001771,0.0,0.0,0.0,0.0,0.0,0.0
2,9990010000001999,0.0,0.0,0.0,0.0,0.0,0.0
3,9990010000008534,0.0,0.0,0.0,0.0,0.0,0.0
4,9990010000031123,0.0,0.0,0.0,0.0,-1.0,0.0
5,9990010000034518,0.0,-1.0,0.0,-1.0,-1.0,0.0
6,9990010000035531,0.0,0.0,0.0,0.0,0.0,0.0
7,9990010000035987,0.0,0.0,0.0,0.0,0.0,0.0
8,9990010000039013,0.0,0.0,-1.0,0.0,0.0,0.0
9,9990010000058600,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
df_re_g_u.shape

(470395, 7)

In [29]:
df_re[df_re['tro_members'] == '9990010000070044']

,tbtdr_created_date,tro_members,tbtdr_tbto_id,rating,created_date
310447,2021-07-11 12:19:58.883000,9990010000070044,7308077,-1.0,2021-07
374406,2021-07-24 09:14:59.787000,9990010000070044,7901678,-1.0,2021-07
496379,2021-07-19 07:42:30.430000,9990010000070044,7645563,-1.0,2021-07
562637,2021-07-21 17:23:54.771000,9990010000070044,7779534,-1.0,2021-07
805945,2021-08-04 20:48:24.571000,9990010000070044,8446295,-1.0,2021-08
1200477,2021-08-11 16:27:29.158000,9990010000070044,8724344,-1.0,2021-08
1759715,2021-09-13 20:18:02.787000,9990010000070044,10284130,5.0,2021-09
1763647,2021-07-23 09:40:02.677000,9990010000070044,7851355,-1.0,2021-07
1948357,2021-09-25 12:44:16.136267,9990010000070044,10767590,NaN,2021-09
2519410,2021-10-16 12:28:56.827824,9990010000070044,11523831,NaN,2021-10


In [ ]:
# HIDE CODE

## USING ORDER
import glob

df_tto = []
for end_date in pd.date_range('2021-07-01', '2022-01-01', freq='M'):
    start_date = end_date.replace(day=1)
    end_date_save = start_date + relativedelta(months=1)
    
    start_date_str = start_date.strftime('%d%b%y')
    end_date_str = end_date_save.strftime('%d%b%y')
    
    print(start_date_str, end_date_str)
    
    pf = '/home/server/gli-data-science/data/trans_oshop/{}_{}.csv'.format(start_date_str, end_date_str)
    df_tto.append(pd.read_csv(pf, sep='\t', dtype='object'))
    
df_tto = pd.concat(df_tto)

df_tto = df_tto[df_tto['TRO_MEMBERS'] != 'N']
df_tto['TRO_DATE'] = pd.to_datetime(df_tto['TRO_DATE'])
df_tto['TRO_NET'] = df_tto['TRO_NET'].astype('float', errors='ignore').astype('int', errors='ignore')


df_tto['TRO_QTY'] = df_tto['TRO_QTY'].astype(int)

In [ ]:
df_tto['TRO_VOUCHER_USAGE'] = df_tto['TRO_VOUCHER_USAGE'].astype('float', errors='ignore').astype('int', errors='ignore')

In [ ]:
df_tto['TRO_DATE_MONTH'] = df_tto['TRO_DATE'].dt.strftime('%Y-%m')

In [ ]:
df_tto_sel = df_tto[(df_tto['TRO_DATE'] >= '2021-08-01') \
            & (df_tto['TRO_DATE'] <= '2021-12-30')]

In [ ]:
df_tto_sel_g = df_tto_sel.groupby(['TRO_MEMBERS','TRO_DATE_MONTH'])\
                        .agg({'TRO_NET':'sum',
                              'TRO_VOUCHER_USAGE':'sum'})

In [ ]:
df_tto_sel_g.head()

In [ ]:
df_tto_sel_g_u = df_tto_sel_g.unstack(level=1).fillna(0).reset_index()

In [ ]:
df_tto_sel_g_u.head(1000)